## 1. 필요 라이브러리 호출

In [4]:
import pandas as pd
import requests
import json

path = "/Users/kook/Desktop/"

## 2. 감성분석 api 호출 및 함수 정의

In [5]:
import requests
import json
import pandas as pd

def Sentiment_API(df, column_name):

    client_id = '5ex4s8ep3j'
    client_secret = 'FOPrm2MzgLrbVzVE0I0fs8oKDzwLEms5bZTdfQ6z'
    url = "https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    sentiments = []

    for content in df[column_name]:
        # 950글자까지로 지정
        # API가 1000까지만 처리됨
        data = {
            "content": content[:950]  
        }
        response = requests.post(url, data=json.dumps(data), headers=headers)
        result = json.loads(response.text)

        highlights = []
        for sentence in result["sentences"]:
            highlights.append(sentence.get("highlights", []))

        sentiments.append({
            "label": result["document"]["sentiment"],
            "confidence": result["document"]["confidence"],
            "highlights": highlights
        })

    sentiments_df = pd.DataFrame(sentiments)
    df = pd.concat([df, sentiments_df], axis=1)
    
    return df

def extract_highlighted_text(df, title_column, highlights_column):
    highlighted_texts = []

    for idx, row in df.iterrows():
        title = row[title_column]
        highlights = row[highlights_column]
        extracted_texts = []

        for highlight_set in highlights:
            for highlight in highlight_set:
                offset = highlight['offset']
                length = highlight['length']
                extracted_texts.append(title[offset:offset + length])
        
        highlighted_texts.append(' '.join(extracted_texts))
    
    df['highlighted_texts'] = highlighted_texts
    return df

In [6]:
# 감성분석 데이터셋
df = pd.read_excel("금산인삼축제 뉴스기사 크롤링.xlsx")
df

,company,url,title,content
0,아시아경제,https://n.news.naver.com/mnews/article/277/000...,"'바가지 없는 지역축제 조성'…관광공사, 백종원과 지역관광 활성화 나선다",더본코리아와 음식 활용 지역관광 활성화 MOU한국관광공사가 백종원 대표와 손잡고 ...
1,뉴스1,https://n.news.naver.com/mnews/article/421/000...,대전·충남,"충남도청 전경,/뉴스1(대전ㆍ충남=뉴스1) = ◇대전시-이장우 시장 10:00 대..."
2,주간조선,https://n.news.naver.com/mnews/article/053/000...,"백종원, 지역 축제 '바가지 요금' 잡으러 간다",백종원 더본코리아 대표. photo 뉴시스지역 먹거리 축제에서의 '바가지 요금'을...
3,뉴시스,https://n.news.naver.com/mnews/article/003/001...,"금산군, 슬로·트렌드 푸드 만들기 등 주민 재능계발 교육","제과제빵, 쌀베이킹 활용 등 8개 교육 추진 금산군 재능계발 전문교육이 주민들로부..."
4,뉴시스,https://n.news.naver.com/mnews/article/003/001...,대전·세종·충남,"[세종=뉴시스] ▲오전 7시 20분, 대전 대덕헤윰포럼, 디딤돌플라자▲오전 7시 ..."
...,...,...,...,...
133,노컷뉴스,https://n.news.naver.com/mnews/article/079/000...,"충남 금산군, 전 직원 금산세계인삼축제 티셔츠 입고 홍보",금산세계인삼축제 홍보 포스터. 금산군 제공충남 금산군(군수 박범인)은 전 직원이 ...
134,노컷뉴스,https://n.news.naver.com/mnews/article/079/000...,"박범인 금산군수, 제41회 금산세계인삼축제 준비 철저 당부",박범인 금산군수. 금산군 제공충남 금산군은 박범인 군수가 14일 군청 상황실에서 ...
135,뉴시스,https://n.news.naver.com/mnews/article/003/001...,"금산세계인삼축제 국제인삼교역전, 400만달러 수출협약",총 7건 협약지난해 130만달러 대비 270만달러 증가 국제인삼교역전 *재판매 및...
136,뉴스1,https://n.news.naver.com/mnews/article/421/000...,"충남도, 지역 대형축제 앞두고 ‘고위험병원체 선제검사’",탄저균·페스트균·브루셀라균·코로나19 등 충남도청 전경./뉴스1(내포=뉴스1) 이...


## 3. 감성분석

In [7]:
df_sentiment = Sentiment_API(df, 'title')

In [8]:
df_with_highlights = extract_highlighted_text(df_sentiment, 'title', 'highlights')

In [9]:
df_sentiment

,company,url,title,content,label,confidence,highlights,highlighted_texts
0,아시아경제,https://n.news.naver.com/mnews/article/277/000...,"'바가지 없는 지역축제 조성'…관광공사, 백종원과 지역관광 활성화 나선다",더본코리아와 음식 활용 지역관광 활성화 MOU한국관광공사가 백종원 대표와 손잡고 ...,positive,"{'negative': 0.016679823, 'positive': 99.61240...","[[{'offset': 23, 'length': 17}]]",백종원과 지역관광 활성화 나선다
1,뉴스1,https://n.news.naver.com/mnews/article/421/000...,대전·충남,"충남도청 전경,/뉴스1(대전ㆍ충남=뉴스1) = ◇대전시-이장우 시장 10:00 대...",neutral,"{'negative': 0.0473967, 'positive': 0.02445237...","[[{'offset': 3, 'length': 2}]]",충남
2,주간조선,https://n.news.naver.com/mnews/article/053/000...,"백종원, 지역 축제 '바가지 요금' 잡으러 간다",백종원 더본코리아 대표. photo 뉴시스지역 먹거리 축제에서의 '바가지 요금'을...,neutral,"{'negative': 4.541487, 'positive': 0.8141132, ...","[[{'offset': 20, 'length': 6}]]",잡으러 간다
3,뉴시스,https://n.news.naver.com/mnews/article/003/001...,"금산군, 슬로·트렌드 푸드 만들기 등 주민 재능계발 교육","제과제빵, 쌀베이킹 활용 등 8개 교육 추진 금산군 재능계발 전문교육이 주민들로부...",neutral,"{'negative': 0.0383008, 'positive': 2.2455974,...","[[{'offset': 8, 'length': 23}]]",트렌드 푸드 만들기 등 주민 재능계발 교육
4,뉴시스,https://n.news.naver.com/mnews/article/003/001...,대전·세종·충남,"[세종=뉴시스] ▲오전 7시 20분, 대전 대덕헤윰포럼, 디딤돌플라자▲오전 7시 ...",neutral,"{'negative': 0.0402412, 'positive': 0.02229116...","[[{'offset': 6, 'length': 2}]]",충남
...,...,...,...,...,...,...,...,...
133,노컷뉴스,https://n.news.naver.com/mnews/article/079/000...,"충남 금산군, 전 직원 금산세계인삼축제 티셔츠 입고 홍보",금산세계인삼축제 홍보 포스터. 금산군 제공충남 금산군(군수 박범인)은 전 직원이 ...,neutral,"{'negative': 0.03224166, 'positive': 0.0307524...","[[{'offset': 8, 'length': 23}]]",전 직원 금산세계인삼축제 티셔츠 입고 홍보
134,노컷뉴스,https://n.news.naver.com/mnews/article/079/000...,"박범인 금산군수, 제41회 금산세계인삼축제 준비 철저 당부",박범인 금산군수. 금산군 제공충남 금산군은 박범인 군수가 14일 군청 상황실에서 ...,neutral,"{'negative': 0.08886535, 'positive': 0.0471470...","[[{'offset': 10, 'length': 22}]]",제41회 금산세계인삼축제 준비 철저 당부
135,뉴시스,https://n.news.naver.com/mnews/article/003/001...,"금산세계인삼축제 국제인삼교역전, 400만달러 수출협약",총 7건 협약지난해 130만달러 대비 270만달러 증가 국제인삼교역전 *재판매 및...,neutral,"{'negative': 0.06320516, 'positive': 0.1220618...","[[{'offset': 18, 'length': 11}]]",400만달러 수출협약
136,뉴스1,https://n.news.naver.com/mnews/article/421/000...,"충남도, 지역 대형축제 앞두고 ‘고위험병원체 선제검사’",탄저균·페스트균·브루셀라균·코로나19 등 충남도청 전경./뉴스1(내포=뉴스1) 이...,neutral,"{'negative': 0.5416572, 'positive': 0.11676115...","[[{'offset': 5, 'length': 24}]]",지역 대형축제 앞두고 ‘고위험병원체 선제검사


## 4. 감성분석 기여단어 추출(TF-IDF vectorizer 활용)

In [10]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

# 형태소 분석기 초기화
okt = Okt()

# highlighted_texts 컬럼을 형태소 단위로 나누기
df_sentiment['tokenized_highlighted_texts'] = df_sentiment['highlighted_texts'].apply(lambda x: ' '.join(okt.nouns(x)))

# TF-IDF 벡터라이저 초기화
vectorizer = TfidfVectorizer()

# TF-IDF 행렬 생성
tfidf_matrix = vectorizer.fit_transform(df_sentiment['tokenized_highlighted_texts'])

# 단어 리스트 추출
terms = vectorizer.get_feature_names_out()

# 제목별로 중요한 단어 1개 추출
def extract_top_word(row):
    tfidf_vector = tfidf_matrix[row.name].toarray().flatten()
    top_index = tfidf_vector.argmax()
    top_word = terms[top_index]
    return top_word

# 각 제목별로 중요한 단어 1개 추출하여 'top_word' 컬럼에 저장
df_sentiment['top_word'] = df_sentiment.apply(extract_top_word, axis=1)

# 필요한 컬럼만 선택하여 최종 데이터프레임 생성
df_final = df_sentiment[['highlighted_texts', 'top_word', 'label', 'confidence']]

# 최종 데이터프레임 출력
print(df_final)

            highlighted_texts top_word     label  \
0           백종원과 지역관광 활성화 나선다      활성화  positive   
1                          충남       충남   neutral   
2                      잡으러 간다       간다   neutral   
3     트렌드 푸드 만들기 등 주민 재능계발 교육       계발   neutral   
4                          충남       충남   neutral   
..                        ...      ...       ...   
133   전 직원 금산세계인삼축제 티셔츠 입고 홍보       직원   neutral   
134    제41회 금산세계인삼축제 준비 철저 당부       당부   neutral   
135               400만달러 수출협약       수출   neutral   
136  지역 대형축제 앞두고 ‘고위험병원체 선제검사       대형   neutral   
137      2023 금산세계인삼축제 후원금 기탁       기탁   neutral   

                                            confidence  
0    {'negative': 0.016679823, 'positive': 99.61240...  
1    {'negative': 0.0473967, 'positive': 0.02445237...  
2    {'negative': 4.541487, 'positive': 0.8141132, ...  
3    {'negative': 0.0383008, 'positive': 2.2455974,...  
4    {'negative': 0.0402412, 'positive': 0.02229116...  
..                               

In [11]:
df_final.to_excel('금산인삼축제 감성분석 최고 기여 단어.xlsx', index=False)